# Подготовка признаков 

<img src="./pict/MLmem.jpg"  
  width="600"
/>

Чтобы предсказать класс, обратимся к знакомой логистической регрессии.

Логистическая регрессия подходит для задачи классификации. Например, как у нас, когда выбор между двумя категориями — потребуется страховая выплата или нет. 
Попробуем обучить нашу модель. Как думаете, получится это сделать на сырых данных? Рискнём.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

data = pd.read_csv('./data/travel_insurance.csv')

In [2]:
data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41


In [3]:
train, valid = train_test_split(data, test_size=0.25, random_state=12345)

features_train = train.drop('Claim', axis=1)
target_train = train['Claim']
features_valid = valid.drop('Claim', axis=1)
target_valid = valid['Claim']

In [66]:
model = LogisticRegression()
model.fit(features_train, target_train) 

Теперь можете сказать: «А я говорил/говорила!» Действительно, ошибка. 

Почему это произошло?

Принадлежность к категории логистическая регрессия вычисляет по формуле, состоящей из признаков. Они могут быть только численные. Наши данные содержали и категориальные признаки тоже — в этом и была ошибка.

## Прямое кодирование

Преобразовать категориальные признаки в численные поможет техника прямого кодирования, или отображения (One-Hot Encoding, OHE).

Принцип работы One-Hot Encoding объясним на значениях признака Gender.

Для каждого значения признака Gender (F, M, None) создаётся столбец:
    
- Gender_F (от англ. female, «женщина»),
- Gender_M (от англ. male, «мужчина»),
- Gender_None (данных о поле нет).

<img src="./pict/10.png"  
  width="600"
/>

Резюмируем. Техникой OHE категориальные признаки переводятся в численные в два этапа:
- Для каждого значения признака создаётся новый столбец;
- Если объекту категория подходит, присваивается 1, если нет — 0.

Новые признаки (Gender_F, Gender_M, Gender_None) называются dummy-variable.

Для прямого кодирования в библиотеке pandas есть функция `pd.get_dummies()` («получить фиктивные переменные»).

## Дамми-ловушка

С прямым кодированием не всё так просто. Когда данных в избытке, можно угодить в ловушку фиктивных признаков. Расскажем, как в неё не попасть.

Чтобы подать документы на визу в США, нужно доказать, что деньги у вас есть. Вы решили перестраховаться, поэтому взяли и выписку с банковского счёта, и справку с работы, и 2-НДФЛ. Хотя визовому центру достаточно двух документов. 

Вашей модели лишняя информация тоже не очень-то нужна. Если оставить всё как есть, обучаться она будет сложнее. 

В таблицу добавились три новых столбца. Поскольку они сильно связаны между собой, один удалим без сожаления. Восстановить столбец можно по оставшимся двум. Так мы не попадём в дамми-ловушку

Столбец уберём вызовом функции `pd.get_dummies()` с аргументом `drop_first`. 

Он удаляет первую колонку и передаётся как `drop_first=True` или `drop_first=False` (True — первый столбец сбрасывается, False — не сбрасывается).

Обучая логистическую регрессию, вы можете столкнуться с предупреждением библиотеки sklearn. Чтобы его отключить, укажите аргумент `solver='liblinear'`

## Порядковое кодирование

Расскажем о другой технике кодирования признаков в решающем дереве и случайном лесе.

Если решающее дерево задаёт вопросы в узлах, значит, может работать и с категориальными признаками? 

<img src="./pict/1.jpg"  
  width="1100"
/>

А теперь попробуем обучить модель: 

In [5]:
# from sklearn.tree import DecisionTreeClassifier

# tree = DecisionTreeClassifier(random_state=12345)
# tree.fit(features_train, target_train) 

Опять ошибка. Вызвана тем, как решающее дерево обучается в библиотеке `sklearn`.

Как это починить? Нужна новая техника, чтобы закодировать цифрами выраженные в тексте категории —  Ordinal Encoding. Она работает так:
- Фиксируется, какой цифрой кодируется класс;
- Цифры размещаются в столбце.

Техника подходит для преобразования признаков в решающем дереве и случайном лесе

<img src="./pict/11.png"  
  width="300"
/>

Преобразование выполняется в три этапа: 
1. Создаём объект этой структуры данных.

2. Чтобы получить список категориальных признаков, вызываем метод fit() — как и в обучении модели. Передаём ему данные как аргумент.

3. Преобразуем данные функцией transform(). Изменённые данные будут храниться в переменной data_ordinal.

Чтобы код добавил названия столбцов, оформим данные в структуру DataFrame():

Если преобразование признаков требуется лишь один раз, как в нашей задаче, код можно упростить вызовом функции fit_transform(). Она объединяет функции: fit() и transform(). 

## Итоги кодирования

Разберёмся, какую кодировку выбрать и почему Ordinal Encoding не подходит для логистической регрессии.

Вы познакомились с двумя техниками кодирования категориальных переменных. Резюмируем:

1. Если все признаки должны стать количественными, подходит техника OHE;
2. Когда все признаки категориальные, и их нужно преобразовать в числа — Ordinal Encoding.

Почему `Ordinal Encoding` не подходит для логистической регрессии? Она всё норовит посчитать по формуле. Если речь идёт о признаке Age, то это разумно, а вот с Gender есть трудности. Например, сложив значения «1» и «0» («женщина» и «мужчина») и разделив на «2», «средний пол» не получить.

<img src="./pict/12.png"  
  width="900"
/>

# Масштабирование признаков

`Диспе́рсия случа́йной величины́ — мера разброса значений случайной величины относительно её математического ожидания`

`Квадратный корень из дисперсии, называется среднеквадратическим отклонением, стандартным отклонением или стандартным разбросом.`

В данных есть столбцы: Age и Commission. 

Допустим, возможен возраст от 0 до 100 лет, а страховая комиссия — от 100 долларов до 1000. 

Значения и их разбросы в столбце Commission больше, поэтому алгоритм автоматически решит, что этот признак важнее возраста. А это не так: все признаки значимы.

Один из методов масштабирования — `стандартизация данных`. 

Предположим, что все признаки распределены нормально, `среднее` (M) и `дисперсия` (D) определяются по выборке. Значения признака преобразуются по формуле:

<img src="./pict/13.png"  
  width="600"
/>

У нового признака устанавливается среднее, равное 0, и дисперсия, равная 1.

В `sklearn` есть отдельная структура для стандартизации данных — `StandardScaler`. Он находится в модуле `sklearn.preprocessing`. 
Импортируем `StandardScaler` из библиотеки:

Создадим объект этой структуры и настроим его на обучающих данных. Настройка — это вычисление среднего и дисперсии:

Преобразуем обучающую и валидационную выборки функцией transform(). Изменённые наборы сохраним в переменных: features_train_scaled и features_valid_scaled:

ри записи изменённых признаков в исходный датафрейм код может вызывать предупреждение `SettingWithCopy`. Причина в особенности поведения `sklearn` и `pandas`. 

Чтобы предупреждение не появлялось, в код добавляют строчку:
`pd.options.mode.chained_assignment = None`

<div class="alert alert-danger">
ВАЖНО! Масштабирование признаков производить только после <b>train_test_split</b>
</div>

# Шпаргалка

In [ ]:
# One-hot-encoding: получение дамми-признаков
pd.get_dummies(df['column'])
pd.get_dummies(df['column'], drop_first=True)
# drop_first = True - удаление первого столбца (избегая дамми-ловушки)

In [ ]:
# Ordinal Encoding

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
encoder.fit(data)
data_ordinal = encoder.transform(data)

# добавление названий столбцов
data_ordinal = pd.DataFrame(encoder.transform(data), columns=data.columns)

# автоматическое обучение и преобразование
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

In [ ]:
# Стандартизация
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df_scaled = scaler.transform(df)

# Accuracy метрика

Предположим мы работаем в патентном бюро. К нам принесли патент что изобрели неинвазивный способ определения лейкоза у младенцев. Он предсказывает предрасположенностьноворожденного к равитию лейкоза с точностью `98%`. 

Как мы сейчас увидим этот тест действительно точен на `98%`! И тем не менее он такой же дурацкий, являясь хорошей иллюстрацией почему Accurasy не используют для оценки точности бинарной классификации.

- Патент предпологает что модель предсказывает лейкоз, если и только пациента зовут Григорий. 

Давайте посмотрим, как мой тест на лейкоз вписывается в эти рамки. В 2022 году в Украине приблизительно 3% младенцам из 1000 дают имя Григорий,а распространенность лейкоза на протяжении жизни составляет 1.4%, или 14 человек на каждую 1000

Если мы считаем, что эти два фактора являются взаимно независимыми и применим мой тест "Григорий означает лейкоз" к 1млн человек, то можно ожидать, что мы увидим матрицу несоотвествий, которая выглядит так:

<div class="scrollable_content">
    <table cellpadding="0" cellspacing="0" style="width: 500px; text-align: center;">
        <thead ><tr>
            <th scope="col" style="text-align: center;"></th>
            <th scope="col" style="text-align: center;">Лейкоз</th>
            <th scope="col" style="text-align: center;">Не лейкоз</th>
            <th scope="col" style="text-align: center;">Всего</th>
               </tr>
        </thead>
        <tbody>
        <tr>
            <td style="text-align: center;">Григорий</td>
            <td style="text-align: center;">42</td>
            <td style="text-align: center;">4 958</td>
            <td style="text-align: center;">5 000</td>
        </tr>
        <tr>
            <td style="text-align: center;">Не Григорий</td>
            <td style="text-align: center;">13 958</td>
            <td style="text-align: center;">981 042</td>
            <td style="text-align: center;">995 000</td>
        </tr>
        <tr>
            <td style="text-align: center;">Всего</td>
            <td style="text-align: center;">14 000</td>
            <td style="text-align: center;">986 000</td>
            <td style="text-align: center;">1 000 000</td>
        </tr>
        </tbody></table><div></div></div>

In [44]:
correct  = 42 + 981042
total = 42+ 4930 + 13958 + 981042

result = correct/total
    
print('result {:.2%}'.format(result))

result 98.11%


# Баланс и дисбаланс классов

Долю правильных ответов, близкую к 100%, мы получили. А вот понимания, что происходит - нет. В нашей задаче наблюдается сильный class imbalance.

Классы несбалансированны, когда их соотношение далеко от 1:1. class balance наблюдается, если их количество примерно равно.

<img src="./pict/14.jpeg"  
  width="400"
/>

`Accuracy` не подходит. Нужна новая метрика! Но прежде — несколько важных определений.

Вы уже знаете, что класс с меткой «1» называется положительным, с меткой «0» — отрицательным.

Если сравнить эти ответы с предсказаниями, получается такое деление:
- True Positive (TP) и True Negative (TN)
- False Positive (FP) и False Negative  (FN).

Резюмируем. Характеристики «положительный» и «отрицательный» относятся к `предсказанию`, а «истинный» и «ложный» — к его `правильности`.

<b>Истинно положительные ответы</b> Что значит истинно положительный ответ (TP)? Модель пометила объект единицей, и его настоящее значение тоже — 1.

<b>Истинно отрицательные ответы</b> Если предсказанное и фактическое значение класса отрицательные, ответ истинно отрицательный.

<b>Ложноположительные ответы</b> Ошибка первого рода — это ложноположительные ответы (FP). Они возникают, когда модель предсказала «1», а вот действительное значение класса — «0».

<b>Истинно положительные ответы</b> Ошибка второго рода — ложноотрицательные ответы (FN). Ложноотрицательные ответы появляются, когда модель предсказала «0», а действительное значение класса — «1».

## Матрица ошибок

Что получаем:

По главной диагонали (от верхнего левого угла) выстроены правильные прогнозы:
- TN в левом верхнем углу;
- TP в правом нижнем углу.

Вне главной диагонали — ошибочные варианты:
- FP в правом верхнем углу;
- FN в левом нижнем углу.

<img src="./pict/15.png"  
  width="500"
/>

Матрица неточностей находится в знакомом модуле `sklearn.metrics`. Функция `confusion_matrix()` принимает на вход верные ответы и предсказания, а возвращает матрицу ошибок.

# Полнота

Матрица ошибок поможет построить новые метрики. Начнём с recall.

Полнота выявляет, какую долю положительных среди всех ответов выделила модель. Обычно они на вес золота, и важно понимать, как хорошо модель их находит. 

Recall рассчитывается по такой формуле:

<img src="./pict/16.png"  
  width="700"
/>

Полнота — это доля TP-ответов среди всех, у которых истинная метка 1. Хорошо, когда значение recall близко к единице: модель хорошо ищет положительные объекты. Если ближе к нулю — модель надо перепроверить и починить.

# Точность

Точность определяет, как много отрицательных ответов нашла модель, пока искала положительные. Чем больше отрицательных, тем ниже точность.

Precision рассчитывается по такой формуле:

<img src="./pict/17.png"  
  width="700"
/>

Напомним, TP — это истинно положительные ответы. FP — отмеченные моделью положительные ответы. Нам нужна точность, близкая к единице.

# Точность против полноты

Когда модель плохо предсказывает положительные классы, то мала как точность, так и полнота. Можно ли повысить их значения?

<img src="./pict/17.jpg"  
  width="600"
/>

In [6]:
# Example

total_row = 1000

# Matrix
#  230  170 
#  200  400

tn = 230
fp = 170
fn = 200
tp = 400

In [7]:
accuracy = (tn + tp) / total_row
accuracy

0.63

In [8]:
recall = tp / (tp + fn)
recall

0.6666666666666666

In [9]:
precision = tp / (tp + fp)
precision

0.7017543859649122

In [10]:
tn = 500
fp = 0
fn = 0
tp = 500

recall = tp / (tp + fn)
recall

1.0

Полноту мы разобрали, а как же достичь высокой точности? 

В формуле учитываются только ошибки для положительного класса, а не отрицательного. Нужно обучить модель, которая, наоборот, отвечает «1» как можно реже. 

`Но отвечать на всё «0» не стоит, так метрику не «взломать»: в формуле ноль будет делиться на ноль.`

# F1-мера

По отдельности полнота и точность не слишком информативны. Нужно одновременно повышать показатели обеих. Или обратиться к новой метрике, которая их объединит.

Полнота и точность оценивают качество прогноза положительного класса с разных позиций. Recall описывает, как хорошо модель разобралась в особенностях этого класса и распознала его. Precision выявляет, не переусердствует ли модель, присваивая положительные метки. 

Важны обе метрики. Контролировать их параллельно помогают агрегирующие метрики, одна из которых — F1-score. Это среднее гармоническое полноты и точности. Единица в F1 означает, что соотношение полноты и точности равно 1:1. 

<img src="./pict/18.png"  
  width="700"
/>

<b>Важно:</b> когда полнота или точность близки к нулю, то к 0 приближается и само среднее гармоническое.

На графике отображены значения F1-меры при разных значениях точности и полноты. Синий цвет соответствует нулю, а жёлтый — единице.

<img src="./pict/19.png"  
  width="500"
/>

# Несбалансированная классификация

Алгоритмы машинного обучения считают все объекты обучающей выборки равнозначными по умолчанию. Если важно указать, что какие-то объекты важнее, их классу присваивается вес (class_weight).

В алгоритме логистической регрессии в библиотеке `sklearn` есть аргумент `class_weight`. По умолчанию он равен `None`, т. е. классы равнозначны:

Если указать `class_weight='balanced'`, алгоритм посчитает, во сколько раз класс «0» встречается чаще класса «1». 

Обозначим это число N (неизвестное количество раз). Новые веса классов выглядят так:

Бóльший вес будет у редкого класса.

Аргумент `class_weight` также есть у решающего дерева и случайного леса.

## Увеличение выборки

Как сделать объекты редкого класса не такими редкими в данных?

Теперь в тесте за решение любой задачи вы получаете 1 балл. Самые важные задачи повторяются по несколько раз, чтобы их легче запомнить. 

Когда обучают модели, такая техника называется `upsampling`.

Преобразование проходит в несколько этапов:
- Разделить обучающую выборку на отрицательные и положительные объекты;
- Скопировать несколько раз положительные объекты;
- С учётом полученных данных создать новую обучающую выборку;

Перемешать данные: идущие друг за другом одинаковые вопросы не помогут обучению.

Скопировать объекты несколько раз поможет синтаксис умножения списков в Python. Чтобы повторить элементы списка, он умножается на число (нужное количество раз):

In [62]:
answers = [0, 1, 0]
print(answers)

answers_x3 = answers * 3
print(answers_x3) 

[0, 1, 0]
[0, 1, 0, 0, 1, 0, 0, 1, 0]


## Уменьшение выборки

Как сделать объекты частого класса не такими частыми?

Вместо повторения важных вопросов убрать часть неважных. Это можно сделать техникой `downsampling`

Преобразование проходит в несколько этапов:
    
- Разделить обучающую выборку на отрицательные и положительные объекты;
- Случайным образом отбросить часть из отрицательных объектов;
- С учётом полученных данных создать новую обучающую выборку;

Перемешать данные. Положительные не должны идти следом за отрицательными: алгоритмам будет сложнее обучаться.

## Порог классификации

Как лучше обучать логистическую регрессию? Посмотрим, что у неё внутри.

Чтобы определить ответ, логистическая регрессия вычисляет, к какому классу близок объект, затем сравнивает результат с нулём. Для удобства близость к классам переведём в вероятность классов: модель пытается оценить, насколько вероятен тот или иной класс. 

У нас всего два класса (ноль и единица). Нам достаточно вероятности класса «1». Число будет от нуля до единицы: если больше 0.5 —  объект положительный, меньше — отрицательный.

Граница, где заканчивается отрицательный класс и начинается положительный, называется порогом (англ. threshold). По умолчанию он равен 0.5, но что если его поменять?

Как поменяется точность и полнота, если уменьшить порог c 0.5 до 0.2?

`Точность уменьшится, а полнота увеличится` почему?

## Изменение порога

В библиотеке `sklearn` вероятность классов вычисляет функция `predict_proba()` . На вход она получает признаки объектов, а возвращает вероятности:

# PR-кривая

Изобразим на графике, как выглядят значения метрик при изменении порога.

На графике по вертикали наносится значение точности, по горизонтали — полноты. 

Кривая, показывающая их значения, называется PR-кривой. Чем выше кривая, тем лучше модель.

<img src="./pict/20.png"  
  width="500"
/>

# TPR и FPR

Когда положительных объектов нет, точность не вычислить. Выберем другие характеристики, в которых нет деления на ноль.

Прежде чем перейти к новой кривой, дадим несколько важных определений. 

Как измерить, насколько правильно классификатор находит объекты? Долей верно предсказанных объектов к общему числу объектов класса. Это отношение называется 

`TPR` (True Positive Rate) или recall. Формула выглядит так, где `P=TP+FN`.

<img src="./pict/21.png"  
  width="700"
/>

Доля ложных срабатываний к общему числу объектов за пределами класса (False Positive Rate, `FPR`) вычисляется аналогично. 

Это отношение `FP-ответов` (False Positives — отрицательные, классифицированные как положительные) к сумме отрицательных ответов: 

`FP` и `TN` (True Negatives — верно классифицированные отрицательные ответы). Ниже дана формула, где `N=FP+TN`:

<img src="./pict/22.png"  
  width="700"
/>

Деления на ноль не будет: в знаменателях значения, которые постоянны и не зависят от изменения модели.

# ROC-кривая

Мы стали свидетелями нового противостояния — `TPR` против `FPR`. Изобразим его на графике.

По горизонтали нанесём долю ложноположительных ответов (`FPR`), а по вертикали — долю истинно положительных ответов (`TPR`). Переберём значения порога логистической регрессии и проведём кривую. 

Она называется `ROC-кривая`, или `кривая ошибок` (receiver operating characteristic; термин пришёл из теории обработки сигналов).

Для модели, которая всегда отвечает случайно, `ROC-кривая` выглядит как прямая, идущая из левого нижнего угла в верхний правый. Чем график выше, тем больше значение `TPR` и лучше качество модели.

<img src="./pict/23.png"  
  width="500"
/>

Чтобы выявить, как сильно наша модель отличается от случайной, посчитаем площадь под ROC-кривой — `AUC-ROC` (Area Under Curve ROC). Это новая метрика качества, которая изменяется от 0 до 1. `AUC-ROC` случайной модели равна 0.5.

Построить ROC-кривую поможет функция `roc_curve()` из модуля `sklearn.metrics`:

На вход она принимает значения целевого признака и вероятности положительного класса. Перебирает разные пороги и возвращает три списка: значения `FPR`, значения `TPR` и `рассмотренные пороги`.

# MSE/RMSE в задаче регрессии

Какая метрика лучше подходит для задч регресии? MSE - это средняя квадратичная ошибка. Но как понять что модель считает верно? 

<font size="5">  
    MSE = sum of squared object errors / number of object
</font> 

<img src="./pict/9.png"  
  width="700" />

Один из простых способов это посмотреть адекватность модели (сравнить значения с средним) 

<img src="./pict/24.png"  
  width="700"
/>

# Коэффициент детерминации

Чтобы не пришлось всё время сравнивать модель со средним, введём новую метрику. Она выражена не в абсолютных значениях, а в относительных.

`Коэффициент детерминации`, или `метрика R2` (coefficient of determination; R-squared), вычисляет долю средней квадратичной ошибки модели от MSE среднего, а затем вычитает эту величину из единицы. Увеличение метрики означает прирост качества модели. 
Формула расчёта R2 выглядит так:

<img src="./pict/25.png"  
  width="700"
/>

- Значение метрики R2 равно единице только в одном случае, если MSE нулевое. Такая модель предсказывает все ответы идеально.
- R2 равно нулю: модель работает так же, как и среднее.
- Если метрика R2 отрицательна, качество модели очень низкое.
- Значения R2 больше единицы быть не может.

# Среднее абсолютное отклонение

Познакомимся с новой метрикой качества — `MAE` (mean absolute error). Она похожа на MSE, но в ней нет возведения в квадрат

Запишем новую метрику символами, а не словами.

<img src="./pict/26.png"  
  width="400"
/>

1. Значение целевого признака для объекта с порядковым номером `i` в выборке, на которой измеряется качество. Например, тестовой. Буква `y` связана с целевым признаком. Нижний индекс показывает номер объекта.

<img src="./pict/27.png"  
  width="400"
/>

2. Значение предсказания для объекта с порядковым номером `i`, например, в тестовой выборке. Знак `circumflexus` над `y` говорит, что это предсказание модели, а не правильный ответ.

Отклонение объекта - это разница значения целевого признака и предсказания 

Чтобы в новой метрике избавиться от разницы между недооценкой и переоценкой, вычисляется `абсолютное отклонение`.

Чтобы собрать отклонения по всей выборке, дополним обозначения:

<img src="./pict/28.png"  
  width="400"
/>

3. Количество объектов в выборке.

<img src="./pict/29.png"  
  width="400"
/>

4. Суммирование по всем объектам выборки (i меняется от 1 до N).

Так мы подошли к формуле `среднего абсолютного отклонения`, или `MAE`:

<img src="./pict/30.png"  
  width="700"
/>

# Интерпретация MAE

Чтобы рассчитать MSE, за константу мы принимали среднее значение. Но подойдёт ли оно для вычисления MAE? Разберёмся.

Константная модель выбирается так, чтобы значение метрики MAE было предельно низким. Нужно найти такое значение `a`, при котором достигается минимум:

<img src="./pict/31.png"  
  width="700"
/>

Минимум получается, когда `a` равно `медиане` целевого признака.

# Влияние разброса на метрики

Рассмотрим, как MAE и RMSE зависят от разброса целевого признака.

В отличие от MAE, метрика RMSE чувствительнее к большим значениям: значимые ошибки сильно влияют на итоговое значение квадратного корня из средней квадратичной ошибки. 

Перед вами три графика распределения ошибок (по горизонтали указаны значения ошибок, а по вертикали — их количество):
    
1. У первого поровну маленьких (от 0 до 10) и больших (40 и выше) ошибок.
2. На втором и третьем графиках постепенно увеличивается разрыв. Возникают очень большие ошибки, становится больше маленьких. Значения MAE не меняются: большие ошибки компенсируются маленькими. А вот RMSE увеличивается.

<img src="./pict/33.jpg"  
  width="700"
/>

Теперь вы знаете, что можно изменять значение одной метрики, не меняя другую.